In [1]:
!pip install -r requirements.txt

  Using cached anyio-4.8.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached asttokens-3.0.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached comm-0.2.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached debugpy-1.8.12-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
  Using cached decorator-5.1.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached executing-2.1.0-py2.py3-none-any.whl.metadata (8.9 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached ipykernel-6.29.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached ipython-8.31.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached jedi-0.19.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cach

In [60]:
! python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 97.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
with open("example_md_to_text.txt", "r", encoding="latin-1") as f: # TO DO: check proper encoding for .md files
    markdown_example = f.read()

In [3]:
from pprint import pprint
import tqdm
# pprint(markdown_example)

Basic OLlama Workflow

In [4]:
# import ollama

In [5]:
# # basic workflow of getting llama embeddings with ollama
# ollama.embeddings(model="llama3.2:3b",
#                 prompt="Hello")

In [6]:
# # basic workflow of generating responses with ollama
# ollama.generate(model="llama3.2:3b",
#                 prompt="Whos is Obama")

Loading HF to get Attention Masks

In [7]:
with open("HF_TOKEN.txt", "r") as f:
    hf_token = f.read()

In [8]:
from huggingface_hub import HfFolder, whoami

HfFolder.save_token(hf_token)
print(whoami()["name"])

alberto-lorente


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [21]:
import torch

device = "cpu"
if torch.cuda.is_available():
  print("Cuda available")
  device = "cuda:0"

Cuda available


In [22]:
# using huggingface tokenizer for attettion layer
# make sure you have GPU enabled
# takes around 5 mins to load with TPUs

from transformers import AutoTokenizer, AutoModel

model_id = "HIT-TMG/KaLM-embedding-multilingual-mini-v1" # which model to use?
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)
model = AutoModel.from_pretrained(model_id)

In [23]:
model.to(device)

Qwen2Model(
  (embed_tokens): Embedding(151936, 896)
  (layers): ModuleList(
    (0-23): 24 x Qwen2DecoderLayer(
      (self_attn): Qwen2SdpaAttention(
        (q_proj): Linear(in_features=896, out_features=896, bias=True)
        (k_proj): Linear(in_features=896, out_features=128, bias=True)
        (v_proj): Linear(in_features=896, out_features=128, bias=True)
        (o_proj): Linear(in_features=896, out_features=896, bias=False)
        (rotary_emb): Qwen2RotaryEmbedding()
      )
      (mlp): Qwen2MLP(
        (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
        (up_proj): Linear(in_features=896, out_features=4864, bias=False)
        (down_proj): Linear(in_features=4864, out_features=896, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
    )
  )
  (norm): Qwen2RMSNorm((896,), eps=1e-06)
  (rotary_emb): Qwen2RotaryEmbedding()
)

- text enriching with VLLM to add a little description of tables before hey appear?

- embeddings that return attention layer

- perform similarity on the attention layer -> cluster similar sentences

- summarize the clusters

- tree-based graph

## Splitting the text into sentences

This method may take a lot of time. Alternatives?

In [61]:
import spacy

nlp = spacy.load("fr_core_news_sm")

In [62]:
doc = nlp(markdown_example)
sents = [sent.text for sent in doc.sents]

In [64]:
sents[0:10]

['# Séance publique\n\n\n\n# du jeudi 28 octobre 2021\n\n\n\n# à 18h00\n\n\n\n# Chorum',
 'Alain Gilles',
 '- Halle Vacheresse\n\n\n\n#',
 'Rue des Vernes à Roanne\n\n\n\n#',
 'PROCES',
 'VERBAL\n\n\n\n',
 "L'an deux mille vingt et un, le 28 octobre à 18 h 00, les conseillers communautaires de Roannais Agglomération, se sont réunis à l\x92Espace Chorum \x96 Halle",
 'Vacheresse \x96',
 'Rue des Vernes à Roanne.\n\n\n\n',
 'La convocation de tous les conseillers a été faite le 22 octobre 2021, dans les formes et délais prescrits par la loi, par Yves Nicolin, Président.\n\n\n\n']

## Getting the text embeddings


### Example for one piece of text

In [24]:
# dir(tokenizer)

In [27]:
tokenized_markdown = tokenizer(markdown_example, return_tensors="pt").to(device)

In [40]:
len(tokenized_markdown[0])

20468

In [50]:
model.eval()
with torch.no_grad():
  # first batch (only one since we are processing one doc)
  # final token
    embeddings = model(**tokenized_markdown)[0][:, 0]

In [52]:
embeddings.shape

torch.Size([1, 896])

We take the last token since it's the one which contains all the aggregate info.

In [54]:
# embeddings

In [55]:
import torch.nn.functional as F

normalized_embeddings = F.normalize(embeddings, p=2, dim=1)

In [56]:
normalized_embeddings

tensor([[-2.5701e-02,  2.9421e-02,  2.3318e-02,  7.8289e-03, -1.4849e-02,
          3.7893e-02, -2.6897e-02,  4.2656e-02,  5.1599e-02, -7.3862e-03,
         -4.0127e-02, -1.8006e-02,  2.8234e-02, -3.1621e-02,  1.7363e-02,
          1.4090e-02, -3.5483e-02,  3.8051e-02,  5.7395e-02,  2.1053e-02,
          1.2293e-02,  2.1901e-02,  7.4137e-02,  5.4875e-02,  9.3957e-03,
         -3.2502e-02,  3.8113e-02, -2.5295e-03, -1.1333e-02, -2.3000e-03,
          6.7924e-03, -7.2779e-03, -3.4598e-02, -3.0293e-03,  1.1351e-02,
          2.4411e-02,  1.5954e-02,  1.9381e-02, -4.4216e-03, -1.0174e-02,
         -5.4419e-03,  1.2559e-02, -5.5566e-02, -3.6830e-02,  4.7059e-03,
         -1.7030e-02,  1.2295e-02, -5.1929e-02,  2.4320e-02,  3.5810e-02,
          4.7526e-03, -1.7858e-02, -2.7658e-02,  2.4482e-02, -2.6175e-02,
         -6.2970e-02, -4.0419e-02,  4.9006e-02,  2.4184e-02,  3.2100e-02,
          2.4963e-02,  7.7604e-02,  4.9397e-02,  2.2441e-03,  1.0605e-01,
          5.1354e-03, -1.5346e-02, -1.

### For a list of sentences

In [75]:
def compute_embeddings(sentence):

    tokenized_sentences = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(device)

    model.eval()
    with torch.no_grad():
        embeddings = model(**tokenized_sentences)[0][:, 0]
        # print(embeddings.shape)

    normalized_embeddings = F.normalize(embeddings, p=2, dim=1)
    detached_embeddings = normalized_embeddings.detach().cpu().numpy() # detached into cpu so that we can manipulate them for clustering

    torch.cuda.empty_cache() # Careful with running out of memory

    return detached_embeddings

In [73]:
detached_embeddings = compute_embeddings(sents[0])

torch.Size([1, 896])


In [76]:
detached_sents_embeddings = [compute_embeddings(sent) for sent in sents]

In [78]:
len(detached_sents_embeddings)

913

In [79]:
len(detached_sents_embeddings) == len(sents)

True

In [80]:
detached_sents_embeddings[0].shape

(1, 896)

## Clustering